In [200]:
# Make cells wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [202]:
import json
import numpy as np
import os

# !pip install pywebarchive
import webarchive


import re
import json
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

nparr = np.array

# Importing and loading data into Pandas DFs

In [203]:
# Load in eval trial data
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

gestureNames = ["Forward flick", "Right flick", "Right tilt", "Left flick", \
                "Left tilt", "Pull close", "Push away", "Turn to right", "Turn to left"]

# Import all webarchives in the directory
files = []
calib_files = []
for r,d,f in os.walk("."):
    for file in f:
        if 'GAZEL dot' in r:
            if ".webarchive" in file:
                files.append(os.path.join(r, file))
            elif "gazel_checkpoint" in file:
                calib_files.append(os.path.join(r, file))


# Webarchive format -> parsed JSON dictionaries
fileData = []
for file in files:
    tmp = webarchive.open(file)
    jsonStr = cleanhtml(tmp._main_resource.data.decode())
    try:
        jsonData = json.loads(jsonStr)
        for key in jsonData.keys():
            jsonData[key] = json.loads(jsonData[key])
        fileData.append(jsonData)
    except:
        print("Failed on " + file)
        
# Load in calibration round data

# 5 rounds, a = [[], [], [], [], []]
# Each round is made up of [x,y] pairs.
#   The x is embeddings
#   The y is dot ground truth
calibData = []
for calib_file in calib_files:
    with open(calib_file, 'r') as f:
        a = json.loads(f.read())
        for i in range(len(a)):
            a[i] = json.loads(a[i])
        calibData.append(a)
        
print("Successfully parsed " + str(len(fileData)) + " trials", end=" ")
print("and " + str(len(calibData)) + " calibration rounds")


Successfully parsed 9 trials and 9 calibration rounds


In [319]:
# Create DFs, making first a list of each trial separately, tagged with user ID and everything
dataList = []
calibList = []

for subNum in range(len(fileData)):
    print("Parsing subject #", subNum, end=", ")
    data = fileData[subNum]
    for evalName in data.keys():
        for gestureBlock in data[evalName]:
            for segment in gestureBlock:
                # Unpack each segment trial
                timestamp, detected, target, histories = segment

                detectedGest = detected[0]
                detectedGaze = detected[1]

                # Get the ground truth gesture and square out
                gestureTarget = target[0]
                gestureName = gestureNames[gestureTarget]
                gazeTarget = target[1]

                # Unpack the histories array
                headsize_hist, embeddings_hist, gazepreds_hist, IMU_hist, \
                                    gestdetect_hist, facevisible_hist = histories

                dataList.append([subNum, evalName, gestureName, gazeTarget, timestamp, 
                                 gestureTarget, gazeTarget, detectedGest, detectedGaze, 
                                 headsize_hist, embeddings_hist, gazepreds_hist, gestdetect_hist, IMU_hist, 
                                     facevisible_hist])

    subjectCalData = calibData[subNum]
    for calRound in subjectCalData:
        x,y = calRound
        for i in range(len(x)):
            calibList.append([subNum, [x[i]],y[i]])




# Create dataframe out of the list
GAZELdata = pd.DataFrame(dataList, columns=['Subject', "Set", "Gesture Target Name", "Gaze Target", "Timestamp",
       "Target Gesture", "Target Gaze", "Detected Gesture", "Detected Gaze",
        "Headsize Hist", "Embeddings Hist", "Gazepreds Hist", "Gestpreds Hist", "IMU Hist", "Face Detection Hist"
                                   ])
print("\nGAZELdata has length after creating DF: ", len(GAZELdata))

# Target gestures are top to bottom
# | 1  5 |
# | 2  6 | 
# | 3  7 |
# | 4  8 |

Parsing subject # 0, Parsing subject # 1, Parsing subject # 2, Parsing subject # 3, Parsing subject # 4, Parsing subject # 5, Parsing subject # 6, Parsing subject # 7, Parsing subject # 8, 
GAZELdata has length after creating DF:  3402


In [320]:
# Adding columns to the eval DF and filtering             RUN CELL ABOVE FIRST
minLength = 10

# Function to filter the embeddings and gaze prediction histories based on movement
def getFirstActivity(lst):
    tmp = next((i for i,x in enumerate(lst) if sum(x) != 0), -1)
    # if tmp == -1: tmp = len(lst)
    return tmp

####### Find index of first motion, then trim each gazepred and embedding history there. 
            # Also remove the first few samples to get rid of reaction time
firstIndices = list(map(getFirstActivity, GAZELdata['Gestpreds Hist']))
for col in ['Gazepreds Hist', 'Embeddings Hist', 'Face Detection Hist']:
        GAZELdata[col] = list(map(lambda x: x[0][4:x[1]], zip(GAZELdata[col],firstIndices)))

#### Remove samples where head is not visible.
indices = [np.argwhere(nparr(x) < .9) for x in GAZELdata['Face Detection Hist']]
for col in ['Gazepreds Hist', 'Embeddings Hist', 'Face Detection Hist']:
        GAZELdata[col] = list(map((lambda x: np.delete(x[0], x[1],axis=0)), zip(GAZELdata[col],indices)))        
        
# Then remove all rows where the length is too short (1 sample is about 100 ms)
GAZELdata = GAZELdata.loc[[len(x) >= minLength for x in GAZELdata['Gazepreds Hist']]]



######## Add grid? and Calib? column
GAZELdata["Grid?"] = [x[0] == "g" for x in GAZELdata["Set"]]
GAZELdata['Calib?'] = False


######### Add target locations in XY
GAZELdata['Gaze Target XY'] = np.zeros((len(GAZELdata), 2)).tolist()
def seg2GridCoords(segNum):
    return [(1 + (segNum-1)//4)/3, 1/8 + (((segNum-1) % 4) )/4]
def seg2ListCoords(segNum):
    return [0.5, 1/12 + (segNum-1)/6]

grids = [x[0]=='g' for x in GAZELdata['Set']]
lists = [not x for x in grids]

# GAZELdata.loc[grids, 'Gaze Target XY'] = pd.Series([seg2GridCoords(x) for x in GAZELdata.loc[grids, 'Gaze Target']])
# GAZELdata.loc[lists, 'Gaze Target XY'] = pd.Series([seg2ListCoords(x) for x in GAZELdata.loc[lists, 'Gaze Target']])
GAZELdata.loc[grids, 'Gaze Target XY'] = GAZELdata.loc[grids, 'Gaze Target'].map(seg2GridCoords)
GAZELdata.loc[lists, 'Gaze Target XY'] = GAZELdata.loc[lists, 'Gaze Target'].map(seg2ListCoords)


######## Add raw XY from base model
GAZELdata['Raw Gazepreds Hist'] = [[y[12:14] for y in x] for x in GAZELdata['Embeddings Hist']]


print("After filtering, GAZELdata has length: ", len(GAZELdata))
# # Plot one of the eye histories for fun
# plt.plot(GAZELdata['Gazepreds Hist'][0])

After filtering, GAZELdata has length:  3272


In [314]:
# clf = models[1]
# clf.fit(train_x, train_y)
voters = [clf.predict(x) for x in testDF['Embeddings Hist']]

In [293]:
# Create calib DF and training DF
def xyToSegGrid(lst):
    x,y = lst
    return int(((y//.25)+1) + 4*(x > .5))

def xyToSegList(lst):
    x,y = lst
    return int(y//(1/6) + 1)

def f(clf, dataset):
    newOutputs = [np.mean(clf.predict(x), axis=0) for x in dataset['Embeddings Hist']]
    print("Testing abs error on same conditions as base model", mean_absolute_error(dataset['Gaze Target XY'].tolist(), newOutputs))
    
    # Accuracy for new model
    predSegs = []
    for i in range(len(newOutputs)):
        curXY = newOutputs[i]
#         print(dataset.iloc[i])
        if dataset.iloc[i]["Set"][0] == "g":
            predSeg = xyToSegGrid(curXY)
        else:
            predSeg = xyToSegList(curXY)
        predSegs.append(predSeg)

        
CALIBdata = pd.DataFrame(calibList, columns=["Subject", "Embeddings Hist", "Gaze Target XY"])
CALIBdata['Set'] = "grid" # Assume grid for accuracy calcs
CALIBdata['Gaze Target'] = CALIBdata['Gaze Target XY'].map(xyToSegGrid)
CALIBdata['Grid?'] = True
CALIBdata['Calib?'] = True


ALLdata = pd.concat([GAZELdata, CALIBdata], sort=False).reset_index()
print("All data length: ", len(ALLdata))

postDF = ALLdata[['Subject', 'Set', 'Calib?', 'Grid?', 'Embeddings Hist', 'Gaze Target XY', 'Gaze Target']].copy()

All data length:  15878


In [228]:
# Functions for getting base accuracy and cm error from a subject
def getBaseErrorCM(subjectData):
    a = np.array(subjectData['Gazepreds Hist'])
    b = [np.mean(x, axis=0) for x in a]
    return mean_absolute_error(subjectData['Gaze Target XY'].to_list(), b)

def getErrorWithModel(subjectData, model):
    a = subjectData['Gazepreds Hist']
    feats = subjectData['Embeddings Hist']
    b = [np.mean(model.predict(x), axis=0) for x in feats]

    return mean_absolute_error(subjectData['Gaze Target XY'].to_list(), b)

def printAcc(suff, correct, total):
    print(suff, str(correct) + "/" + str(len(gazePred)) + " = " + str(round(correct/len(gazePred)*100,2)) + "%")

def getBaseAccuracy(subData):
    gazeGT = subData['Target Gaze']
    gazePred = subData['Detected Gaze'].to_numpy().astype(int)
    correct = sum(gazeGT == gazePred)
    printAcc("Unprocessed Gaze Accuracy:", correct, len(gazePred))
    
    gestGT = subData['Target Gesture']
    gestPred = subData['Detected Gesture']
    correct = sum(gestGT == gestPred)
    printAcc("Unprocessed Gestures Accuracy:", correct, len(gestPred))

getBaseAccuracy(GAZELdata)
# Add per-square metrics to this

Unprocessed Gaze Accuracy: 1308/3312 = 39.49%
Unprocessed Gestures Accuracy: 2440/3312 = 73.67%


In [251]:
# Check to see if face is in frame 
fd = []
gp = []
for i in GAZELdata.index:
    fd.extend(GAZELdata.loc[i, 'Face Detection Hist'])
    gp.extend(GAZELdata.loc[i, 'Gazepreds Hist'])
a = pd.Series(fd)
a[a<.9]
print("TODO: Remove trials where face is not in frame from testing and training\n\n\n\n\n")

TODO: Remove trials where face is not in frame from testing and training







# The Machine Begins to Learn

In [300]:
# List of classifier models
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import mean_absolute_error,accuracy_score
import scipy
mode = scipy.stats.mode

import copy
dcopy = copy.deepcopy

# Classification
models = [
    KNeighborsClassifier(n_neighbors=len(files)),
    ExtraTreesClassifier(),
    ExtraTreesClassifier(n_estimators=400),
    RandomForestClassifier(),
    RandomForestClassifier(n_estimators=400),
    SGDClassifier(),
    LogisticRegression()
]

#### Splitting data into pairs
def getPairsFromTable(tbl, together=False): 
    c = []
    gridOrList = []
    gazeTargets = []
    for i in tbl.index:
        embeds = tbl.loc[i, 'Embeddings Hist']
        XY = tbl.loc[i, 'Gaze Target XY']
#         faceVis = tbl.loc[i, 'Face Detection Hist']
        if together:
            c.append([embeds[-6:],XY])
            gazeTargets.append(tbl.loc[i, 'Gaze Target'])
            gridOrList.append(tbl.loc[i, 'Set'])
        else:
            for j,sample in enumerate(embeds): # remove samples where face is not visible
#                 if faceVis[j] > .9:
                c.append([sample, XY])
                gazeTargets.append(tbl.loc[i, 'Gaze Target'])
                gridOrList.append(tbl.loc[i, 'Set'])
    return c,gazeTargets, gridOrList


In [ ]:
# sum(GAZELdata['Face Detection Hist'] < .9)
a = []
c = 0
for thing in GAZELdata['Face Detection Hist']:
    if (sum(thing)/len(thing)) < 1.0:
        print(thing)
        c += 1
    a.extend(thing)

In [294]:
len(a)


51224

In [266]:
# len(getPairsFromTable(GAZELdata)[0])

50686

In [305]:
# Classification 
# GAZELdata, CALIBdata, ALLdata, postDF
def classifier_SingleEmbedThenVote(clf, trainingDF, testingDF):
    train, trainGazetargs, trainSet = getPairsFromTable(trainingDF, together=False)
    train_x = [np.array(x[0]).flatten() for x in train]
    train_y = trainGazetargs

    print("Fitting...")
    clf.fit(train_x, train_y)
    
    # Test on the testing data
    print("Predicting...")
    voters = [clf.predict(x) for x in testingDF['Embeddings Hist']]
    voteOutputs = [mode(x).mode[0] for x in voters]
    preds = np.array(voteOutputs)
    gt = testingDF['Gaze Target'].to_numpy()
    print("score", sum(preds == gt)/len(preds))
    print("model", clf)
    return sum(preds == gt)/len(preds)


classDF = GAZELdata.copy()
split = 0.8
trainDF = classDF.sample(frac = split)
testDF = classDF.drop(trainDF.index).reset_index()
actualTrainDF = pd.concat([CALIBdata, trainDF], sort=False).reset_index()

model = ExtraTreesClassifier(n_estimators = 100)
classifier_SingleEmbedThenVote(model, actualTrainDF, testDF)
# classifier_SingleEmbedThenVote(model, trainDF, testDF)

Fitting...
Predicting...
score 0.5587786259541985
model ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)


0.5587786259541985

In [193]:
postDF


,Subject,Set,Calib?,Grid?,Embeddings Hist,Gaze Target XY,Gaze Target
0,0,grid1_results,False,True,"[[-0.82373046875, 0.472412109375, -0.977539062...","[0.3333333333333333, 0.375]",2
1,0,grid1_results,False,True,"[[-0.7900390625, 0.6982421875, -1.171875, -0.3...","[0.6666666666666666, 0.125]",5
2,0,grid1_results,False,True,"[[0.3671875, 0.74609375, -1.2763670682907104, ...","[0.6666666666666666, 0.875]",8
3,0,grid1_results,False,True,"[[-0.3227539360523224, 0.2541504204273224, -1....","[0.3333333333333333, 0.125]",1
4,0,grid1_results,False,True,"[[-0.329345703125, 0.8007813096046448, -1.3789...","[0.3333333333333333, 0.875]",4
5,0,grid1_results,False,True,"[[0.1773681789636612, 0.3701172173023224, -1.1...","[0.6666666666666666, 0.625]",7
6,0,grid1_results,False,True,"[[-1.2744139432907104, 1.4599608182907104, -1....","[0.3333333333333333, 0.625]",3
7,0,grid1_results,False,True,"[[-0.9033203721046448, 0.4975586235523224, -1,...","[0.6666666666666666, 0.375]",6
8,0,grid1_results,False,True,"[[-1.1386717557907104, 1.0869139432907104, -1....","[0.6666666666666666, 0.625]",7
9,0,grid1_results,False,True,"[[-0.60546875, 0.71875, -1.3125, -0.380859375,...","[0.3333333333333333, 0.625]",3


In [109]:
# subData = GAZELdata
# gazeGT = subData['Target Gaze'].to_numpy()
# gazePred = subData['Detected Gaze'].to_numpy().astype(int)


In [110]:
sum(gazeGT == gazePred)

1308